In [1]:
runs -y rm %

In [2]:
rm -f runs.db .runsrc

# Starting out

We begin by creating a new `.runsrc` file and creating our database `runs.db`. Running any `runs` command will do this. Note that without `--assume-yes`, this would prompt the user for confirmation. You can also use the short flag `-y`.

In [3]:
runs --assume-yes ls

Config not found. Using default config:
{'args': '',
 'db_path': '/Users/ethan/lab-notebook/demo/runs.db',
 'dir_names': '',
 'root': '/Users/ethan/lab-notebook/demo/.runs'}



In [4]:
ls runs.db  # database that maintains metadata about runs

runs.db


In [ ]:
cat .runsrc  # config file

[main]
root : /Users/ethan/lab-notebook/demo/.runs
db_path : /Users/ethan/lab-notebook/demo/runs.db
dir_names : 
args : 



The `.runsrc` establishes various defaults for the `runs` command although these can be overriden with command line arguments. We will explore some of the capabilities of this file later in the tutorial.

# Creating runs

In [ ]:
runs -y new --path=demo --command=python --description="Demonstrate lab-notebook capabilities."

Path:
demo
Description:
Demonstrate lab-notebook capabilities.
Command sent to session:
python
List active:
tmux list-session
Attach:
tmux attach -t demo



In [ ]:
runs ls demo

demo


You can query the database about runs:

In [ ]:
runs lookup command demo

demo: python


In [ ]:
runs lookup datetime demo

demo: 2018-12-25T13:24:30.012028


You can create multiple runs with a single run command:

In [ ]:
runs -y new --path=demo --command="python -c 'print(1)'" \
            --path=demo2 --command="python -c 'print(2)'" \
            --description="Demonstrate lab-notebook capabilities."

In [ ]:
runs ls dem%

In [ ]:
tmux ls | grep demo

Note that `demo` has been overwritten. Previously the command for `demo` was `python`. Now it is

In [ ]:
runs lookup command demo

In [ ]:
runs ls --active

You can also filter by time.

In [ ]:
runs ls --since $(date "+%Y-%m-%d")

In [ ]:
runs ls --from-last 1day

For info on accepted formats, run `runs ls -h` (ommitted for brevity).

All these selection mechanisms are available to other `runs` subcommands (`rm`, `mv`, `ls`, `lookup`, `change-description`, `reproduce`, `correlate`, `kill`).

# Changing runs

In [ ]:
runs -y mv demo demo1

In [ ]:
runs ls --active

In [ ]:
tmux ls

In [ ]:
runs change-description --active 'A new description'

In [ ]:
runs lookup description --active

We can also kill the TMUX session for runs without deleting the database record:

In [ ]:
runs -y kill --active

In [ ]:
tmux ls | grep demo  # no runs found

In [ ]:
runs ls --active  # no runs found

In [ ]:
runs ls demo%  # but runs were not deleted from database

# Using the `.runsrc`

In [ ]:
runs -y rm %

In [ ]:
rm -rf .runs .runsrc

In this section we will focus on two fields in the `.runsrc`:
* `dir_names` specifies directories that will be placed inside `root` and will be synchronized with run paths (created, moved, and deleted with them).
* `args` specifies flags that should be passed to every run. The `<path>` keyword gets replaced with the path of the run.

In [ ]:
echo '[main]
root : /Users/ethan/lab-notebook/demo/.runs
db_path : /Users/ethan/lab-notebook/demo/runs.db
dir_names : write-dir
args : --write-path=${main:root}/write-dir/<path>/hello.txt' > .runsrc

In [ ]:
cat file_io_demo.py

In [ ]:
runs -y new --path=demo1 --command='python3 file_io_demo.py' \
            --path=demo2 --command='python3 file_io_demo.py' \
            --description='Demonstrate .runsrc capabilities'

Note that the `--write-path` arg has been passed to each run with the value specified in `.runsrc`. Also note that the directory `.runs/write-dir` was created by the `runs` command (because of the `dir_names` section in `.runsrc`), not by `demo_script.py`:

In [ ]:
sleep 1 && tree .runs/write-dir/

In [ ]:
runs -y mv demo2 demo0

`.runs/write-dir/demo2` and all its contents have been moved to `.runs/write-dir/demo0`:

In [ ]:
tree .runs/write-dir/

Finally let's see what happens when we delete runs.

In [ ]:
runs -y rm demo0

`.runs/write-dir/demo0` and all its contents have been removed.

In [ ]:
tree .runs/write-dir/

# Using specs

In [ ]:
runs -y rm %

In [ ]:
rm -rf .runsrc runs.db

In [ ]:
runs -y new --path=demo1 --command="python dummy_script.py --flag --arg=1 --arg=2" \
            --path=demo2 --command="python dummy_script.py --arg=1" \
            --description="Demonstrate spec capabilities"

In [ ]:
runs to-spec --active > run.json

In [ ]:
cat run.json

Note that `arg2` is a list of lists. This is how specs represent repeated args.

In [ ]:
runs -y from-spec run.json --path=from-spec-demo --description="Demonstrate the use of specs to generate runs."